In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

# ==============================
# 1. Load images
# ==============================
big_map = np.array(Image.open(r"D:\RPL\Tiles\Mashhad\satellite\18\173888\102307.png").convert("RGB"))
small_map = np.array(Image.open(r"D:\RPL\Tiles\Mashhad\thermal\18_173888_102307.png").convert("RGB"))

# ==============================
# 2. Read 8 numbers from Excel (one row)
# ==============================
df = pd.read_excel("js_excels/predicted-mashhad-gpu.xlsx")

row = df.iloc[0]


# ==============================
# 3. SOURCE points (small map corners)
# ==============================
h, w, _ = small_map.shape

src_pts = np.array([
    [w, h],        # bottom-right
    [0, h],         # bottom-left
    [0, 0],        # top-left
    [w, 0],        # top-right
], dtype=np.float32)

# ==============================
# 4. DESTINATION points (big map pixels)
# ==============================
dst_pts = np.array([
    [row["x1"], row["y1"]],
    [row["x2"], row["y2"]],
    [row["x3"], row["y3"]],
    [row["x4"], row["y4"]],
], dtype=np.float32)

# ==============================
# 5. Compute homography
# ==============================
H, _ = cv2.findHomography(src_pts, dst_pts)

# ==============================
# 6. Warp small map into big map space
# ==============================
warped_small = cv2.warpPerspective(
    small_map,
    H,
    (big_map.shape[1], big_map.shape[0])
)

# ==============================
# 7. Overlay
# ==============================
mask = np.any(warped_small != [0, 0, 0], axis=2)

result = big_map.copy()
result[mask] = warped_small[mask]

# ==============================
# 8. Visualize
# ==============================
plt.figure(figsize=(12, 12))
plt.imshow(result)
plt.axis("off")
plt.title("Correct Placement Using 8 Big-Map Pixels")
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import os
import math

# ==============================
# CONFIG
# ==============================
EXCEL_PATH = "js_excels/predicted-dehat.xlsx"
NUM_IMAGES = 30          # None = all rows
GRID_COLS = 5           # images per row

# ==============================
# Load Excel
# ==============================
df = pd.read_excel(EXCEL_PATH)
if NUM_IMAGES is not None:
    df = df.head(NUM_IMAGES)

results = []  # (overlay_image, title)

# ==============================
# Process each row
# ==============================
for idx, row in df.iterrows():

    sat_path = row["sat"]
    th_path = row["th"]

    if not os.path.exists(sat_path) or not os.path.exists(th_path):
        print(f"[SKIP] Missing file at row {idx}")
        continue

    # Load images
    big_map = np.array(Image.open(sat_path).convert("RGB"))
    small_map = np.array(Image.open(th_path).convert("RGB"))

    h, w, _ = small_map.shape

    # ==============================
    # Source points (thermal image)
    # Order MUST match training:
    # TL, TR, BL, BR
    # ==============================
    src_pts = np.array([
        [0, 0],   # top-left
        [w, 0],   # top-right
        [0, h],   # bottom-left
        [w, h],   # bottom-right
    ], dtype=np.float32)

    # ==============================
    # Destination points (satellite)
    # x1..x4 follow same order
    # ==============================
    dst_pts = np.array([
        [row["x1"], row["y1"]],  # TL
        [row["x2"], row["y2"]],  # TR
        [row["x3"], row["y3"]],  # BL
        [row["x4"], row["y4"]],  # BR
    ], dtype=np.float32)

    # ==============================
    # Compute homography
    # ==============================
    H, _ = cv2.findHomography(src_pts, dst_pts)
    if H is None:
        print(f"[FAIL] Homography failed at row {idx}")
        continue

    # ==============================
    # Warp thermal into satellite
    # ==============================
    warped = cv2.warpPerspective(
        small_map,
        H,
        (big_map.shape[1], big_map.shape[0])
    )

    # ==============================
    # Overlay warped thermal
    # ==============================
    overlay = big_map.copy()
    mask = np.any(warped != [0, 0, 0], axis=2)
    overlay[mask] = warped[mask]

    # ==============================
    # Draw predicted quadrilateral
    # (visual order: TL → TR → BR → BL)
    # ==============================
    quad_pts = np.array([
        [row["x1"], row["y1"]],  # TL
        [row["x2"], row["y2"]],  # TR
        [row["x4"], row["y4"]],  # BR
        [row["x3"], row["y3"]],  # BL
    ], dtype=np.int32)

    cv2.polylines(
        overlay,
        [quad_pts],
        isClosed=True,
        color=(255, 0, 0),
        thickness=3
    )

    results.append((
        overlay,
        f"{idx}: {os.path.basename(th_path)}"
    ))

# ==============================
# Grid visualization
# ==============================
n = len(results)
rows = math.ceil(n / GRID_COLS)

plt.figure(figsize=(5 * GRID_COLS, 5 * rows))

for i, (img, title) in enumerate(results):
    plt.subplot(rows, GRID_COLS, i + 1)
    plt.imshow(img)
    plt.title(title, fontsize=9)
    plt.axis("off")

plt.tight_layout()
plt.show()
